# MODEL Notebook

In [1]:

import numpy as np
import random
import pandas as pd
import torch
from torch.utils import data
import os   
#from generator import Dataset_CSV
from torch.utils.data import DataLoader


def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f


class Dataset_CSV_train(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, location):
        'Initialization'
        self.locationX = location + '/data/'
        self.locationY = location + '/labels/'
        files = listdir_nohidden(self.locationX)
        self.list_IDs = list(filter(lambda f: f.endswith('.csv'), files))
        
    def __len__(self):
        'Denotes the total number of samples'
        #return 96
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
    
        ID = self.list_IDs[index]

        # Load data and get label
        
        df_data = pd.read_csv(self.locationX + ID)
        df_labels = pd.read_csv(self.locationY + ID)        
        size = df_data.shape[0]
        
        #for cqt
        minsize = 2048
        #for mfcc
        #minsize = 5520
        
        try:
            position = random.randrange(0, size - minsize) #The start of the slice
        except ValueError:
            position = 0
            
        X = np.empty((minsize, 5, df_data.shape[1])) #5520 because its the smallest song size
        Y = np.zeros((minsize,88))

        auxiliary = 0

        for j in range(position, position+minsize - 5):

            if j == position:
                for k in range(5):
                    #Creating the 5 windows image
                    X[auxiliary][k] = df_data.iloc[j + k]
                if j < (position + minsize - 2):
                    #Creating the labels, taking the label of the mid window
                    Y[auxiliary] = df_labels.iloc[j + 2]

                auxiliary+=1
            else:
                for k in range(5):
                    if k < 3:
                      #Copying the values of the previous image to avoid data repetition
                      X[auxiliary][k] = X[auxiliary - 1][k + 1]
                    else:
                        #print(X.shape)
                        #
                        #print(k)
                        #print(j)
                        #
                        #print(df_data.shape)
                        X[auxiliary][k] = df_data.iloc[j + k]


                if j < (position + minsize - 2):
                    #Creating the labels, taking the label of the mid window
                    Y[auxiliary] = df_labels.iloc[j + 2]

                auxiliary+=1

        #Normalise X using min-max normalisation
        
        #For filterbanks
        #max_value = 120.1003100275508
        #min_value = -330.7445963332333
        
        #For MFCC
        #max_value = 841.837813253034
        #min_value = -831.0670773337255
        
        #For CQT
        max_value = 41.775738
        min_value = 0
        
        x_norm = (X  - min_value) / (max_value - min_value)
        x_norm = np.reshape(x_norm,(minsize,1, 5, df_data.shape[1]))
        
        
        #for i,row in enumerate(Y):
        #    yes = np.where(row==1)[0]
        #    for element in yes:
        #        if element -1 >= 0:
        #            Y[i,element-1] = 1
        #        if element +1 <= 87:
        #            Y[i,element+1] = 1
        
        return x_norm, Y


class Dataset_CSV_test(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, location):
        'Initialization'
        self.locationX = location + '/data/'
        self.locationY = location + '/labels/'
        files = listdir_nohidden(self.locationX)
        self.list_IDs = list(filter(lambda f: f.endswith('.csv'), files))
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
    
        ID = self.list_IDs[index]

        # Load data and get label
        
        df_data = pd.read_csv(self.locationX + ID)
        df_labels = pd.read_csv(self.locationY + ID)        
        size = df_data.shape[0]
        
        #for mfcc
        #position = 3500 #The shortest song has 9263 windows, so if all the songs start at the window 3500
                         #We are avoiding the start of the song when there's nothing being played.
        #minsize = 5520
        
        #for CQT
        position = 172
        minsize = 2048
        
        
        X = np.empty((minsize, 5, df_data.shape[1])) #minsize because its the smallest song size
        Y = np.zeros((minsize,88))

        auxiliary = 0

        for j in range(position, position+minsize -5):

            if j == position:
                for k in range(5):
                    #Creating the 5 windows image
                    X[auxiliary][k] = df_data.iloc[j + k]
                if j < (position + minsize - 2):
                    #Creating the labels, taking the label of the mid window
                    Y[auxiliary] = df_labels.iloc[j + 2]

                auxiliary+=1
            else:
                for k in range(5):
                    if k < 3:
                      #Copying the values of the previous image to avoid data repetition
                      X[auxiliary][k] = X[auxiliary - 1][k + 1]
                    else:
                        X[auxiliary][k] = df_data.iloc[j + k]

                if j < (position + minsize - 2):
                    #Creating the labels, taking the label of the mid window
                    Y[auxiliary] = df_labels.iloc[j + 2]

                auxiliary+=1

        #Normalise X using min-max normalisation
        
        #For filterbanks
        #max_value = 120.1003100275508
        #min_value = -330.7445963332333
        
        #For MFCC
        #max_value = 841.837813253034
        #min_value = -831.0670773337255
        
        #For CQT
        max_value = 41.775738
        min_value = 0
        
        x_norm = (X  - min_value) / (max_value - min_value)
        x_norm = np.reshape(x_norm,(minsize,1, 5, df_data.shape[1]))
        
        #for i,row in enumerate(Y):
        #    yes = np.where(row==1)[0]
        #    for element in yes:
        #        if element -1 >= 0:
        #            Y[i,element-1] = 1
        #        if element +1 <= 87:
        #            Y[i,element+1] = 1
        #
        
        return x_norm, Y


In [2]:
trainGenerator = Dataset_CSV_train("finalDatasetCQT/train/")
trainloader = DataLoader(trainGenerator, batch_size=16,
                        shuffle=True, num_workers=4)

testGenerator = Dataset_CSV_test("finalDatasetCQT/test/")
testloader = DataLoader(testGenerator, batch_size=2,
                        shuffle=True, num_workers=2)

In [3]:
x, y = trainGenerator[0]

In [4]:
x.shape

(512, 1, 5, 384)

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import sigmoid


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(1,3))
        self.conv1_bn = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3,3))
        self.conv2_bn = nn.BatchNorm2d(16)
        
        self.conv3 = nn.Conv2d(16, 16, kernel_size=(1,3))
        self.conv3_bn = nn.BatchNorm2d(16)
        
        self.pool = nn.MaxPool2d((1, 2))
        self.drop = torch.nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(3*46*16, 256)
        self.fc2 = nn.Linear(256, 88)

    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        #print("1 ", x.shape)
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.drop(self.pool(F.relu(self.conv3_bn(self.conv3(x)))))
        #print("2 ", x.shape)
        x = x.view(-1, 3*46*16)
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return sigmoid(x)

    
class shortNet(nn.Module):
    def __init__(self):
        super(shortNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(1,5), stride=1)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3))
        
        self.pool1 = nn.MaxPool2d((1, 2), stride = (1,2))
        self.pool2 = nn.MaxPool2d((1, 2))
        self.drop = torch.nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(64*3*94, 88)
        self.tanh = torch.nn.Tanh()

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.tanh(x)
        x = self.drop(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.tanh(x)
        x = self.drop(x)
        x = self.pool2(x)
        x = x.view(-1, 64*3*94)
        x = self.fc1(x)
        return sigmoid(x)    

class longNet(nn.Module):
    def __init__(self):
        super(longNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3,3), padding=1)
        self.conv1_bn = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=(3,3))
        self.conv2_bn = nn.BatchNorm2d(16)
        self.dropbig = torch.nn.Dropout(p=0.5)
        self.dropsmall = torch.nn.Dropout(p=0.25)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(1,3))
        self.conv3_bn = nn.BatchNorm2d(32)
        
        self.pool = nn.MaxPool2d((1, 2))
       
        self.linear1 = nn.Linear(32*3*46, 64)
        self.linear1_bn = nn.BatchNorm1d(64)
        self.linear2 = nn.Linear(64, 128)
        self.linear2_bn = nn.BatchNorm1d(128)
        self.linear3 = nn.Linear(128, 88)


    def forward(self, x):
        #print("0 ", x.shape)
        
        #print("XX0: ", x)
        x = self.conv1(x)
        #print("XX1: ", x)
        x = F.relu(x)
        #print("XX2: ", x)
        #x = self.conv1_bn(x)
        #print("XX3: ", x)
        x = self.pool(x)
        #print("XX4: ", x)
        x = self.dropsmall(x)
        #print("XX5: ", x)
        #print("1: ", x)
        #print("1 ", x.shape)
        x = self.conv2(x)
        x = F.relu(x)
        #x = self.conv2_bn(x)
        x = self.dropsmall(self.pool(x))
        #print("2: ", x)
        #print("2 ", x.shape)
        x = self.dropsmall(self.pool(F.relu(self.conv3_bn(self.conv3(x)))))
        #print("3: ", x)
        #print("3 ", x.shape)
        x = x.view(-1, np.prod(x.shape[1:]))
        x = self.dropsmall(F.relu(self.linear1_bn(self.linear1(x))))
        #print("4: ", x)
        x = self.dropbig(F.relu(self.linear2_bn(self.linear2(x))))
        #print("5: ", x)
        x = self.linear3(x)
        #print("6: ", x)
        return sigmoid(x)



net = shortNet()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
device = 'cpu'
net = net.to(device)

print(repr(net))

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

shortNet(
  (conv1): Conv2d(1, 32, kernel_size=(1, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (drop): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=18048, out_features=88, bias=True)
  (tanh): Tanh()
)


In [ ]:
#ShortNet
loss_values = []
val_accs = []

for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    aux = 0
    net.train() # pytorch way to make model trainable.
    print("Training")
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        for j in range(inputs.shape[0]):
            X = inputs[j].float().to(device)
            Y = labels[j].float().to(device)
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(X)
            #print(outputs)
            loss = criterion(outputs, Y)
            
            loss.backward()
            optimizer.step()
            
            loss_values.append(loss.item())


            # print statistics
            running_loss += loss.item()
            if aux % 20 == 19:    # print every 10 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
                loss_values.append(running_loss / 20)
                running_loss = 0.0
            aux += 1  


    
    test_loss = []
    test_accuracy = []
    predicted_correctly = 0
    total = 0
    aux = 0
    print("Testing")
    with torch.no_grad():
        net.eval()
        for i, data in enumerate(testloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
        
            for j in range(inputs.shape[0]):
                X = inputs[j].float().to(device)
                Y = labels[j].float().to(device)

                output = net(X)
                loss = criterion(output, Y)
                test_loss.append(loss.item())
                #print("Validation loss: ", loss.item())

                ##_, max_indices = torch.max(output.data,1, keepdim=True)
                #max_indices = np.where(output.cpu() > 0.55, 1, 0)
                #n = np.prod(max_indices.shape) #max_indices.size(0) #index 0 for extracting the # of elements
                #train_acc = (torch.tensor(max_indices) == Y).sum()/n
                #test_accuracy.append(train_acc.item())
                
                max_indices = np.where(output.cpu() > 0.55, 1, 0)

                for i, row in enumerate(Y):
                    for j, col in enumerate(row):
                        
                        if max_indices[i][j].item() == 1:
                            #Predicted ones
                            aux += 1
                        if col.item() == 1:
                            #Label with a one
                            total+= 1
                        if col.item() == 0 and max_indices[i][j].item() == 0:
                            continue
                        elif col.item() == 1 and max_indices[i][j].item() == 1:
                            #print("PREDICTED CORRECTLY")
                            predicted_correctly += 1
                        else:
                            continue
                
                
                test_acc = predicted_correctly/total
                test_accuracy.append(test_acc)
    
    print("##########")
    print("EPOCH ", epoch)
    print("##########")
    
    print("##TRAINING STATS##\n")
    print("Loss: ", sum(loss_values)/ len(loss_values))
    #print("Acc: ", sum(test_accuracy)/ len(test_accuracy))
    print("######\n")
    
    print("##VALIDATION STATS##\n")
    print("Loss: ", sum(test_loss)/ len(test_loss))
    acc = sum(test_accuracy)/ len(test_accuracy)
    print("Acc: ", acc)
    val_accs.append(acc)
    print("Predicted correctly: ", predicted_correctly, "out of: ", total)
    print("Numbers of ones predicted in total ", aux)
    print("######\n")
    torch.save(net, "LastModel.pt")
    
print('Finished Training')

Training


In [12]:
#ShortNet
loss_values = []
val_accs = []

for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    aux = 0
    net.train() # pytorch way to make model trainable.
    print("Training")
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        for j in range(inputs.shape[0]):
            X = inputs[j].float().to(device)
            Y = labels[j].float().to(device)
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(X)
            #print(outputs)
            loss = criterion(outputs, Y)
            
            loss.backward()
            optimizer.step()
            
            loss_values.append(loss.item())


            # print statistics
            running_loss += loss.item()
            if aux % 20 == 19:    # print every 10 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
                loss_values.append(running_loss / 20)
                running_loss = 0.0
            aux += 1  


    
    test_loss = []
    test_accuracy = []
    predicted_correctly = 0
    total = 0
    aux = 0
    print("Testing")
    with torch.no_grad():
        net.eval()
        for i, data in enumerate(testloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
        
            for j in range(inputs.shape[0]):
                X = inputs[j].float().to(device)
                Y = labels[j].float().to(device)

                output = net(X)
                loss = criterion(output, Y)
                test_loss.append(loss.item())
                #print("Validation loss: ", loss.item())

                ##_, max_indices = torch.max(output.data,1, keepdim=True)
                #max_indices = np.where(output.cpu() > 0.55, 1, 0)
                #n = np.prod(max_indices.shape) #max_indices.size(0) #index 0 for extracting the # of elements
                #train_acc = (torch.tensor(max_indices) == Y).sum()/n
                #test_accuracy.append(train_acc.item())
                
                max_indices = np.where(output.cpu() > 0.55, 1, 0)

                for i, row in enumerate(Y):
                    for j, col in enumerate(row):
                        
                        if max_indices[i][j].item() == 1:
                            #Predicted ones
                            aux += 1
                        if col.item() == 1:
                            #Label with a one
                            total+= 1
                        if col.item() == 0 and max_indices[i][j].item() == 0:
                            continue
                        elif col.item() == 1 and max_indices[i][j].item() == 1:
                            #print("PREDICTED CORRECTLY")
                            predicted_correctly += 1
                        else:
                            continue
                
                
                test_acc = predicted_correctly/total
                test_accuracy.append(test_acc)
    
    print("##########")
    print("EPOCH ", epoch)
    print("##########")
    
    print("##TRAINING STATS##\n")
    print("Loss: ", sum(loss_values)/ len(loss_values))
    #print("Acc: ", sum(test_accuracy)/ len(test_accuracy))
    print("######\n")
    
    print("##VALIDATION STATS##\n")
    print("Loss: ", sum(test_loss)/ len(test_loss))
    acc = sum(test_accuracy)/ len(test_accuracy)
    print("Acc: ", acc)
    val_accs.append(acc)
    print("Predicted correctly: ", predicted_correctly, "out of: ", total)
    print("Numbers of ones predicted in total ", aux)
    print("######\n")
    torch.save(net, "LastModel.pt")
    
print('Finished Training')

Training
[1,     2] loss: 0.142
[1,     3] loss: 0.137
[1,     4] loss: 0.121
[1,     5] loss: 0.162
[1,     7] loss: 0.152
[1,     8] loss: 0.116
[1,     9] loss: 0.140
[1,    10] loss: 0.147
[1,    12] loss: 0.140
[1,    13] loss: 0.142
[1,    14] loss: 0.152
[1,    15] loss: 0.141
[1,    17] loss: 0.138
[1,    18] loss: 0.133
[1,    19] loss: 0.138
[1,    20] loss: 0.118
Testing
##########
EPOCH  0
##########
##TRAINING STATS##

Loss:  0.13869537137798033
######

##VALIDATION STATS##

Loss:  0.8395478129386902
Acc:  0.07310341515656367
Predicted correctly:  456 out of:  10810
Numbers of ones predicted in total  35840
######

Training
[2,     2] loss: 0.151
[2,     3] loss: 0.134
[2,     4] loss: 0.139
[2,     5] loss: 0.144
[2,     7] loss: 0.126
[2,     8] loss: 0.136
[2,     9] loss: 0.136
[2,    10] loss: 0.147
[2,    12] loss: 0.123
[2,    13] loss: 0.152
[2,    14] loss: 0.138
[2,    15] loss: 0.133
[2,    17] loss: 0.138
[2,    18] loss: 0.124
[2,    19] loss: 0.134
[2,    20]

In [ ]:
loss_values

In [ ]:
loss_values = []

for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    aux = 0
    #net.train() # pytorch way to make model trainable.
    print("Training")
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        for j in range(inputs.shape[0]):
            X = inputs[j].float()
            Y = labels[j].float()
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(X)
            
            loss = criterion(outputs, Y)
            
            loss.backward()
            optimizer.step()
            loss_values.append(loss.item())

            #print("The loss is: ", loss.item())

            # print statistics
            running_loss += loss.item()
            if aux % 20 == 19:    # print every 10 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
                loss_values.append(running_loss / 20)
                running_loss = 0.0
            aux += 1  


    
    test_loss = []
    test_accuracy = []
    print("Testing")
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            for j in range(inputs.shape[0]):
                X = inputs[j].float()
                Y = labels[j].float()

                output = net(X)
                loss = criterion(output, Y)
                test_loss.append(loss.item())
                #print("Validation loss: ", loss.item())

                ##_, max_indices = torch.max(output.data,1, keepdim=True)
                #max_indices = np.where(output.cpu() > 0.55, 1, 0)
                #n = np.prod(max_indices.shape) #max_indices.size(0) #index 0 for extracting the # of elements
                #train_acc = (torch.tensor(max_indices) == Y).sum()/n
                #test_accuracy.append(train_acc.item())
                
                max_indices = np.where(output.cpu() > 0.5, 1, 0)
                predicted_correctly = 0
                total = 0
                for i, row in enumerate(Y):
                    for j, col in enumerate(row):
                        if col.item() == 1:
                            total+= 1
                        if col.item() == 0 and max_indices[i][j].item() == 0:
                            continue
                        elif col.item() == 1 and max_indices[i][j].item() == 1:
                            #print("PREDICTED CORRECTLY")
                            predicted_correctly += 1
                        else:
                            continue
                train_acc = predicted_correctly/total
                test_accuracy.append(train_acc)
    
    print("##########")
    print("EPOCH ", epoch)
    print("##########")
    
    print("##TRAINING STATS##\n")
    print("Loss: ", sum(loss_values)/ len(loss_values))
    #print("Acc: ", sum(test_accuracy)/ len(test_accuracy))
    print("######\n")
    
    print("##VALIDATION STATS##\n")
    print("Loss: ", sum(test_loss)/ len(test_loss))
    print("Acc: ", sum(test_accuracy)/ len(test_accuracy))
    print("Predicted correctly: ", predicted_correctly, "out of: ", total)
    print("######\n")
    
print('Finished Training')

In [ ]:
torch.save(net, "LastModel47percent.pt")

In [ ]:
#_, max_indices = torch.max(output.data,1, keepdim=True)
max_indices = np.where(output.cpu() > 0.55, 1, 0)
n = np.prod(max_indices.shape) #max_indices.size(0) #index 0 for extracting the # of elements
train_acc = (torch.tensor(max_indices) == Y and not (torch.tensor(max_indices) == 0 and Y == 0)).sum()/n
train_acc.item()

In [ ]:
torch.tensor((1,20))

In [ ]:
max_indices = np.where(output.cpu() > 0.5, 1, 0)
total = 0
for i, row in enumerate(Y):
    for j, col in enumerate(row):
        if col.item() == 0 and max_indices[i][j].item() == 0:
            continue
        elif col.item() == 1 and max_indices[i][j].item() == 1:
            total += 1
        else:
            continue

In [ ]:
total

In [ ]:
not(torch.tensor(max_indices) == 0 and Y == 0)

In [ ]:
df = pd.read_csv("finalDataset/train/data/1727.csv")

# KERAS PART

In [ ]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Conv2D,Conv1D, MaxPooling2D, Activation, LSTM, Reshape, Convolution2D, BatchNormalization, Dense, Dropout, Flatten


model = Sequential()


model.add(Conv2D(8, kernel_size=(1,3),
                 activation='relu',
                 input_shape=(5,384,1)))
model.add(MaxPooling2D(pool_size=(1, 2)))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(Dropout(0.25))
model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

model.add(Conv2D(16, (1, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))


#model.add(Dropout(0.25))
#model.add(Conv2D(16, (3, 3)))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(1, 2)))
#model.add(Dropout(0.25))
#model.add(Conv2D(32, (1, 3)))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(1, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(64))
#model.add(Dropout(0.25))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(88, activation='sigmoid'))

print(model.summary())

In [22]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(1,5), strides=(1,1),
    activation='tanh',
    input_shape=(5,384,1)))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(1,2), strides=(1,2)))
model.add(Conv2D(64, (3,3), activation='tanh'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(1,2)))

#model.add(Flatten())
##model.add(Dense(64, activation='sigmoid'))
#model.add(Dense(num_classes, activation='sigmoid'))

print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 5, 380, 32)        192       
_________________________________________________________________
dropout_7 (Dropout)          (None, 5, 380, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 5, 190, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 188, 64)        18496     
_________________________________________________________________
dropout_8 (Dropout)          (None, 3, 188, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 3, 94, 64)         0         
Total params: 18,688
Trainable params: 18,688
Non-trainable params: 0
__________________________________________________

In [21]:
int('5ads')

ValueError: invalid literal for int() with base 10: '5ads'